In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
%cd /content/drive/MyDrive/hse_nlp/

/content/drive/MyDrive/hse_nlp


In [5]:
import pandas as pd
import numpy as np

# 1. Данные
Возьмем [датасет](https://www.kaggle.com/harishcscode/all-news-articles-from-home-page-media-house) с новостями CNN с Кэггла

Посмотрим на него:


In [6]:
data = pd.read_csv('cnn.csv', index_col=0)

In [8]:
data

,KEYWORDS,SUMMARY,TEXT,TITLE,URL
0,"['energy', 'sugars', 'bars', 'grams', 'syrup',...","Story highlights Don't be fooled by the word ""...","Story highlights Don't be fooled by the word ""...",Are energy bars healthy?,https://www.cnn.com/2017/08/25/health/energy-b...
1,"['facebook', 'whats', 'world', 'unfolds', 'tam...",Chat with us in Facebook Messenger.\nFind out ...,Chat with us in Facebook Messenger. Find out w...,Tamagotchi is back,http://www.cnn.com/videos/cnnmoney/2017/10/10/...
2,"['jedi', 'shots', 'rey', 'force', 'wars', 'sta...","ESPN's ""Monday Night Football"" had bears, viki...","ESPN's ""Monday Night Football"" had bears, viki...",'Star Wars: The Last Jedi' trailer debuts on '...,http://money.cnn.com/2017/10/09/media/star-war...
3,"['clients', 'art', 'science', 'scent', 'collid...","Lyn Harris' independent space, Perfumer H , in...","This feature is part of ' Details ,' a new ser...",Art and science collide in this one-of-a-kind ...,https://www.cnn.com/style/article/details-perf...
4,"['akufoaddo', 'tanker', 'incidents', 'dozens',...",(CNN) A tanker exploded near a gas station in ...,(CNN) A tanker exploded near a gas station in ...,"Seven killed, dozens injured in Ghana tanker e...",https://www.cnn.com/2017/10/08/africa/ghana-ta...
...,...,...,...,...,...
938,"['facebook', 'actor', 'whats', 'movie', 'world...",Chat with us in Facebook Messenger.\nFind out ...,Chat with us in Facebook Messenger. Find out w...,Police take shot at actor on movie set,https://www.cnn.com/videos/us/2017/10/04/polic...
939,"['races', 'danelle', 'mph', 'blind', 'life', '...",Photos: Blind skier puts her life in her husba...,(CNN) Danelle Umstead can't see when she skis ...,Blind skier races up to 70 mph,https://www.cnn.com/2016/12/16/health/turning-...
940,"['knew', 'harvey', 'harassed', 'hollywood', 'a...",Journalist Lauren Sivan says Hollywood mogul H...,Journalist Lauren Sivan says Hollywood mogul H...,Reporter accuses Weinstein of sexual advances,https://www.cnn.com/videos/entertainment/2017/...
941,"['cookies', 'continuing', 'information', 'term...",By continuing to browse our site you agree to ...,By continuing to browse our site you agree to ...,Photos of the Times Square even locals love,http://www.cnn.com/travel/gallery/photos-times...


Как видим, дла каждой статьи в датасете присутствует:
- список из ~10 ключевых слов
- саммари
- собственно текст статьи
- заголовок
- ссылка



Поскольку нам не нужен очеь большой корпус, возьмем первые 10 статей и посмотрим на количество токенов:

In [9]:
dataset = data[:10]

In [10]:
dataset.TEXT.apply(lambda x: len(x.split())).sum()

5718

Ура, 6к токенов нам как раз

# 2. Разметка ключевых слов

### 2.1 Теперь разметим ключевые слова самостоятельно:

In [ ]:
words = []
for ind, text in dataset.iterrows():
    print(text['TEXT'])
    print(text['KEYWORDS'])
    keywords = input('Keywords: ').split(', ')
    words.append(keywords)
    print()
    print()

In [19]:
dataset['MY_KEYWORDS'] = words

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
dataset

,KEYWORDS,SUMMARY,TEXT,TITLE,URL,MY_KEYWORDS
0,"['energy', 'sugars', 'bars', 'grams', 'syrup',...","Story highlights Don't be fooled by the word ""...","Story highlights Don't be fooled by the word ""...",Are energy bars healthy?,https://www.cnn.com/2017/08/25/health/energy-b...,"[energy bar, energy, bar, nutrition, food, sna..."
1,"['facebook', 'whats', 'world', 'unfolds', 'tam...",Chat with us in Facebook Messenger.\nFind out ...,Chat with us in Facebook Messenger. Find out w...,Tamagotchi is back,http://www.cnn.com/videos/cnnmoney/2017/10/10/...,"[facebook, messenger, facebook messenger, worl..."
2,"['jedi', 'shots', 'rey', 'force', 'wars', 'sta...","ESPN's ""Monday Night Football"" had bears, viki...","ESPN's ""Monday Night Football"" had bears, viki...",'Star Wars: The Last Jedi' trailer debuts on '...,http://money.cnn.com/2017/10/09/media/star-war...,"[star wars, trailer, film, skywalker, jedi, re..."
3,"['clients', 'art', 'science', 'scent', 'collid...","Lyn Harris' independent space, Perfumer H , in...","This feature is part of ' Details ,' a new ser...",Art and science collide in this one-of-a-kind ...,https://www.cnn.com/style/article/details-perf...,"[perfumer, clients, world, scent, harris, form..."
4,"['akufoaddo', 'tanker', 'incidents', 'dozens',...",(CNN) A tanker exploded near a gas station in ...,(CNN) A tanker exploded near a gas station in ...,"Seven killed, dozens injured in Ghana tanker e...",https://www.cnn.com/2017/10/08/africa/ghana-ta...,"[ghana, explosion, explode, gas, station, gas ..."
5,"['spanish', 'independence', 'regions', 'meets'...","Carles Puigdemont, the President of Catalonia,...",(CNN) Pro-independence Catalans gathered on th...,Catalans' future on line as parliament meets,https://www.cnn.com/2017/10/10/europe/cataloni...,"[catalonia, region, international mediation, m..."
6,"['press', 'excuse', 'secretary', 'trump', 'hou...","""I think it's fake news, but if he did that, I...",(CNN) In a Forbes magazine interview published...,The Trump White House's 'joke' excuse,http://www.cnn.com/2017/10/10/politics/trump-j...,"[trump, joke, iq, joking, secretary, president..."
7,"['pollution', 'repeal', 'kellogg', 'asthma', '...",President Barak Obama shakes Camryn Kellogg's ...,(CNN) The days when all three of her children ...,Health impact of Trump environmental repeal,https://www.cnn.com/2017/10/10/health/health-e...,"[kellog, asthma, child, air, air quality, clea..."
8,"['look', 'response', 'force', 'trump', 'tour',...",(CNN) Finally lumbering into a devastated Puer...,"Michael D'Antonio is the author of the book "" ...",Trump in Puerto Rico: A narcissist's tour de f...,https://www.cnn.com/2017/10/03/opinions/trump-...,"[trump, puerto rico, puerto, rico, president, ..."
9,"['okunoin', 'tohoku', 'japan', 'risshakuji', '...","(CNN) — Upon hearing I would have to climb 1,0...",\n\n\n\nThis article was first published in Ju...,"Yamadera Risshakuji in Tohoku: 1,015 steps to ...",https://www.cnn.com/travel/article/yamadera-te...,"[temple, japan, yamadera, risshakuju, 1015, st..."


Уберем лишние колонки:

In [ ]:
dataset.drop(columns=['TITLE', 'URL'], inplace=True)

Ну и сохраним полученные данные:

In [25]:
dataset.to_csv('cnn_kw.csv', index=False)

### 2.2 Сравним разметки

In [11]:
dataset = pd.read_csv('cnn_kw.csv')

In [12]:
dataset['KEYWORDS'] = dataset['KEYWORDS'].apply(lambda x: set(eval(x)))
dataset['MY_KEYWORDS'] = dataset['MY_KEYWORDS'].apply(lambda x: set(eval(x)))

Если посмотрим на несколько примеров разметки, то увидим несколько вещей:

1. В оригинальном датасете представлены только односложные слова, я же размечала и частотные словосочетания:    


In [13]:
print('ORG:', dataset.KEYWORDS.apply(lambda x: np.mean([len(i.split()) for i in x])).mean())
print('MY:', dataset.MY_KEYWORDS.apply(lambda x: np.mean([len(i.split()) for i in x])).mean())

ORG: 1.0
MY: 1.1366880341880339


2. В оригинальной разметке в качетсве частотных слов многда выделены такие вещи, которые обыяно не хочется считать за ключевые слова.. Вот например тут есть `whats`:

In [14]:
dataset.KEYWORDS[1]

{'chat',
 'facebook',
 'happening',
 'messenger',
 'tamagotchi',
 'unfolds',
 'whats',
 'world'}

В целом, случаев 2 мало, а выделенные автоматически слова довольно хорошие, так что в качестве эталонной разметки возьмем объединение моей и оригинальной: так получим и словосочетания и прелести разнообразности:

In [15]:
dataset = dataset.assign(target=lambda x: x.KEYWORDS.values | x.MY_KEYWORDS.values)

In [16]:
dataset.head()

,Unnamed: 0,KEYWORDS,SUMMARY,TEXT,MY_KEYWORDS,target
0,0,"{energy, grams, sugars, bars, bar, saturated, ...","Story highlights Don't be fooled by the word ""...","Story highlights Don't be fooled by the word ""...","{energy, sugars, bar, saturated, calories, sat...","{energy, grams, contain, fat, snacks, food, su..."
1,1,"{messenger, happening, facebook, chat, unfolds...",Chat with us in Facebook Messenger.\nFind out ...,Chat with us in Facebook Messenger. Find out w...,"{messenger, facebook, chat, world, facebook m...","{messenger, happening, facebook, chat, unfolds..."
2,2,"{rey, shots, trailer, debuts, football, film, ...","ESPN's ""Monday Night Football"" had bears, viki...","ESPN's ""Monday Night Football"" had bears, viki...","{trailer, franchise, red, film, jedi, skywalke...","{football, red, star, jedi, wars, skywalker, r..."
3,3,"{perfumer, art, harris, h, world, quite, oneof...","Lyn Harris' independent space, Perfumer H , in...","This feature is part of ' Details ,' a new ser...","{perfumer, art, perfume, harris, world, client...","{perfumer, perfume, world, oneofakind, science..."
4,4,"{streetsgovernment, nana, dozens, tanker, inci...",(CNN) A tanker exploded near a gas station in ...,(CNN) A tanker exploded near a gas station in ...,"{explode, government, killed, gas, incident, g...","{nana, dozens, tanker, explode, incidents, gas..."


# 3. Автоматическое извлечение ключевых слов

Сначала предобработаем тексты:    
- приведем все в нижний регистр
- затокенизируем 

(для английского языка лемметизация или стемминг не очень продуктивны, так что обойдемся этим)

In [17]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

nltk.download(['punkt', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
STOPS = stopwords.words('english')

In [19]:
def preprocess(text):
    return ' '.join([token.lower() for token in word_tokenize(text)])

In [20]:
dataset['tokenized'] = dataset.TEXT.apply(preprocess)

## 3.1 TF-IDF

Сначала возьмем самый простой вариант -- TF-IDF

Воспользуемся [этой](https://github.com/kavgan/nlp-in-practice/blob/master/tf-idf/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb) имплементацией, но заменим вывод topn ключевых слов, на топ слов, скор которых превосходит порог

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


cv = CountVectorizer(max_df=0.85, stop_words=STOPS, max_features=10000, ngram_range=(1,3))
word_count_vector = cv.fit_transform(dataset.tokenized.tolist())

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [22]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, tol=0.1):
    """get the feature names and tf-idf score of items"""
    
    sorted_items = [item for item in sorted_items if item[1] > tol]  

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [23]:
def tfidf_keywords(text, tol=0.1):
    feature_names = cv.get_feature_names()
    tf_idf_vector = tfidf_transformer.transform(cv.transform([text]))
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    keywords = extract_topn_from_vector(feature_names, sorted_items, tol)

    return set(keywords.keys())

Возьмем два порога 0.1 и 0.15, на них кажется получаем оптимальное количество ключевых слов:

In [24]:
dataset['tfidf_1'] = dataset.tokenized.apply(tfidf_keywords, tol=0.1)
dataset['tfidf_15'] = dataset.tokenized.apply(tfidf_keywords, tol=0.15)

In [25]:
print('tol=0.15:', dataset.tfidf_15[0])
print('tol=0.1:', dataset.tfidf_1[0])

tol=0.15: {'saturated fat', 'energy', 'bars', 'bar', 'saturated', 'fat'}
tol=0.1: {'saturated fat', 'energy', 'energy bars', 'grams', 'bars', 'bar', 'saturated', 'fat', 'contain', 'nutrition', 'protein'}


## 3.2 TextRank

Возьмем имплементацию [отсюда](https://github.com/summanlp/textrank)

In [26]:
!pip install --quiet summa

In [27]:
from summa.textrank import textrank


dataset['textrank'] = dataset.TEXT.apply(lambda x: set(textrank(x, summarize_by=None).split('\n')))

In [28]:
dataset.textrank[0]

{'convenient source',
 'created',
 'equal',
 'meal',
 'meals',
 'replacement aim',
 'rice syrup',
 'satisfy',
 'satisfying',
 'story highlights'}

### 3.3 BERT

Посмотрим на то, как с извлечением ключевых слов справляется трансформер 🌝

Снова не будем изобретать велосипед и воспользуемся вот [этой имплементацией](https://github.com/MaartenGr/KeyBERT)


In [29]:
!pip install --quiet keybert

In [30]:
from keybert import KeyBERT


kw_model = KeyBERT()

Почему-то при указании ngram-range всегда ищутся только нграммы максимальной длины, а хочется разнообразия, поэтому сделаем так:

In [31]:
def bert_keywords(text, tol=0.4):
    kw = []
    for i in range(1,4):
        words = kw_model.extract_keywords(text, keyphrase_ngram_range=(i,i), stop_words=STOPS, top_n=10)
        kw.extend([word[0] for word in words if word[1] > tol])
    return set(kw)

In [32]:
dataset['bert_4'] = dataset['tokenized'].apply(bert_keywords, tol=0.4)
dataset['bert_5'] = dataset['tokenized'].apply(bert_keywords, tol=0.5)

# 4. Шаблоны

Выделим шаблоны, которые встречаются в таргете:

In [33]:
!pip install --quiet spacy==3.1
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.6 MB 69 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [35]:
import spacy

nlp = spacy.load('en_core_web_sm')

Опишем шаблоны:

In [36]:
import re


NP = '(?:ADJ |NOUN |PROPN |VERB |ADP )*(?:NOUN|PROPN)'  # "стандартные" именные группы + штуки типа secretary of state, imported goods и тд
VP = '(?:VERB |PRON |DET )*VERB'  # глагольные группы + штуки как whats и thats
AP = '(?:ADJ |ADV )*ADJ'  # прилагательные и адъективные группы
OT = 'NUM|ADP'  # другие вещи, не входящие в описанное выше, например цифры (ADP потому что iq размечается spacy как ADP)

In [37]:
def check_pattern(text: str):
    doc = nlp(text)
    pos = ' '.join([token.pos_ for token in doc])
    
    patterns = []
    for pattern in [NP, VP, AP, OT]:
        match = re.findall(pattern, pos)
        if not match:
            continue
        for p in match:
            if len(p.split()) == len(pos.split()):
                return text
       

def filter_kw(kw: set):
    new_words = [check_pattern(word) for word in kw]
    return set([w for w in new_words if w])

Проверим, что фильтрация работает:

In [38]:
(dataset.tfidf_1.apply(filter_kw) == dataset.tfidf_1).all()

False

Ура, что-то отфильтровалось

# 5. Оценка автоматически извлеченных слов



In [39]:
def evaluate(y_pred, y_true):

    y_pred = y_pred.apply(lambda x: set([''.join(w.split()) for w in x])).values  ## потому что там в оригинальных ключевых словах есть вещи типа oneofakind
    y_true = y_true.apply(lambda x: set([''.join(w.split()) for w in x])).values

    tp = np.apply_along_axis(func1d=lambda x: len(x[0]),
                             axis=1,
                             arr=np.expand_dims(y_pred & y_true, 1))

    fp = np.apply_along_axis(func1d=lambda x: len(x[0]),
                             axis=1,
                             arr=np.expand_dims(y_pred - y_true, 1))
    
    fn = np.apply_along_axis(func1d=lambda x: len(x[0]),
                             axis=1,
                             arr=np.expand_dims(y_true - y_pred, 1))

    macro_precision = (tp / (tp + fp + 1e-25)).mean()
    macro_recall = (tp / (tp + fn)).mean()
    macro_f1 =  (2 / (macro_recall ** -1 + macro_precision ** -1)).mean()

    tp = tp.sum()
    fp = fp.sum()
    fn = fn.sum()

    micro_precision = tp / (tp + fp) 
    micro_recall = tp / (tp + fn)
    micro_f1 =  2 / (micro_recall ** -1 + micro_precision ** -1)

    results = {'precision': {'macro': macro_precision,
                             'micro': micro_precision},
               'recall': {'macro': macro_recall,
                          'micro': micro_recall},
               'f1': {'macro': macro_f1,
                      'micro': micro_f1}
               }

    return pd.DataFrame(results)

### 5.1 TF-IDF

In [40]:
# без шаблонов
evaluate(dataset.tfidf_1, dataset.target)

,precision,recall,f1
macro,0.761291,0.462754,0.575617
micro,0.688679,0.445122,0.540741


In [41]:
# с шаблонами
evaluate(dataset.tfidf_1.apply(filter_kw), dataset.target)

,precision,recall,f1
macro,0.797960,0.462754,0.585793
micro,0.768421,0.445122,0.563707


In [42]:
# без шаблонов
evaluate(dataset.tfidf_15, dataset.target)

,precision,recall,f1
macro,0.918333,0.249496,0.392386
micro,0.720000,0.219512,0.336449


In [43]:
# с шаблонами
evaluate(dataset.tfidf_15.apply(filter_kw), dataset.target)

,precision,recall,f1
macro,0.941667,0.249496,0.394475
micro,0.857143,0.219512,0.349515


### 5.2 TextRank

In [44]:
# без шаблонов
evaluate(dataset.textrank, dataset.target)

,precision,recall,f1
macro,0.132236,0.219391,0.165012
micro,0.161290,0.243902,0.194175


In [45]:
# с шаблонами
evaluate(dataset.textrank.apply(filter_kw), dataset.target)

,precision,recall,f1
macro,0.148934,0.213835,0.175579
micro,0.174888,0.237805,0.201550


### 5.3 BERT

In [46]:
# без шаблонов
evaluate(dataset.bert_4, dataset.target)

,precision,recall,f1
macro,0.121515,0.173353,0.142878
micro,0.119816,0.158537,0.136483


In [47]:
# с шаблонами
evaluate(dataset.bert_4.apply(filter_kw), dataset.target)

,precision,recall,f1
macro,0.194549,0.173353,0.183340
micro,0.187050,0.158537,0.171617


In [48]:
# без шаблонов
evaluate(dataset.bert_5, dataset.target)

,precision,recall,f1
macro,0.018750,0.024013,0.021058
micro,0.030303,0.024390,0.027027


In [49]:
# с шаблонами
evaluate(dataset.bert_5.apply(filter_kw), dataset.target)

,precision,recall,f1
macro,0.029762,0.024013,0.026580
micro,0.049383,0.024390,0.032653


# 6. Выводы и ошибки автоматического выделения:

### 1. О методах:

- Лучше всего с задачей в такой постановке справляется **TF-IDF**, что логично, учитывая, что такой алгоритм выдедения ключевых слов аналогичен ручному их выделению: простому записыванию самых частотных слов => и вывод **TF-IDF** наиболее похож на написанный вручную таргетный список слов, особенно с применением фильтрации по шаблонам

- Затем следует алгоритм **TextRank**, хотя качество алгоритма сильно хуже, чем у TF-IDF, это как кажется можно объяснить используемыми метриками: точность, полнота и f-мера -- метрики каечства классификации, и поэтому не учитывают ранжирование ответов, в то время как алгоритм **TextRank** все же предназначен для ранжирования, а значит его работу более корректно оценивать при помощю метрик ранжирования (т.е. например `precision@k`, `recall@k`, `f1@k`)

- Хуже всего с задачей справляется **BERT**, причем качество совсем плохое( Но это тоже можно объяснить немного странной работой алгоритма (в частности с `ngram_range`), что пришлось решать костылем => вывод не совсем точный (об этом в пункте 2)

### 2. Качественная оценка:  
Пункт 0: фильтрация по шаблонам помогает улучшить качество еа всех методах (хоть и незначительно), потому что алгоритмы выделяют сочетания слов, опираясь на частотность, а не смысл и логику, отчего получаются странные сочетания (см. ниже)


**TF-IDF** в целом работает нормально, если смотреть на точность, что говорит о том, что он выделяет в целом логичные ключевые слова, однако полноста и f-мера желают лучшего, кажется, стоит посмотреть на порог вхождения ключевого слова в финальный список, в целом видно, что алгоритм выделяет намного меньше ключевых слов, чем, наверное, хотелось бы. Это, кмк, в том числе связано с тем, что в нашем случае корпус документов, на котором мы обучали наш векторайзер, очень маленький, поэтому некоторые слова получают довольно низкие скоры. Так что тут надо посмотреть на больший корпус текстов, чтобы понять, что поменять.

Но **TF-IDF**, очевидно, выделяет наиболее похожие на выделенные человеком ключевые слова, и если посмотреть на вывод, то увидим это:

In [50]:
print('TARGET:', dataset.target[0])
print('PRED:', dataset.tfidf_1[0])

TARGET: {'energy', 'grams', 'contain', 'fat', 'snacks', 'food', 'sugars', 'bars', 'bar', 'saturated', 'calories', 'syrup', 'nutrition', 'saturated fats', 'healthy', 'protein', 'energy bar'}
PRED: {'saturated fat', 'energy', 'energy bars', 'grams', 'bars', 'bar', 'saturated', 'fat', 'contain', 'nutrition', 'protein'}


**TextRank** выделяет больше слов, которые мне кажутся не совсем ключевыми: так в тексте 0 говорится про `energy bars`, в то время как алгоритм не выдает ни самого словосочетания, ни `energy`, ни `bars`, зато выделяет `rice syrup`, `story highlights` и другие слова и сочетания, не являющиеся самыми важными для описания текста.

Не очень понятно, как это решать, поскольку эти результаты в целом кажутся странными, учитывая, что алгоритм работает на совтречаемости слов, а выделенные слова не очень частотные сами по себе (во всяком случае, как показалось мне), может быть проблема в конкретно этой имплементации..

In [51]:
print('TARGET:', dataset.target[0])
print('PRED:', dataset.textrank[0])

print()
print(dataset.SUMMARY[0])

TARGET: {'energy', 'grams', 'contain', 'fat', 'snacks', 'food', 'sugars', 'bars', 'bar', 'saturated', 'calories', 'syrup', 'nutrition', 'saturated fats', 'healthy', 'protein', 'energy bar'}
PRED: {'created', 'rice syrup', 'story highlights', 'replacement aim', 'convenient source', 'meal', 'meals', 'satisfy', 'satisfying', 'equal'}

Story highlights Don't be fooled by the word "energy"Some energy bars contain as much saturated fat as a Snickers bar(CNN) Energy bars are a convenient source of nutrition and come in a wide variety of flavors to satisfy different palates.
But, like many foods in a specific category, not all energy bars are created equal.
For example, some bars covered in chocolate contain as much saturated fat as a Snickers bar; others contain almost as much sugar.
Energy bars containing mostly fruit and nuts can serve as satisfying snacks.
In general, try to aim for bars with less than 3 grams of saturated fat and at least 4 grams of fiber.


**BERT** в отличие от остальных методов опирается не на статистику, а семантику текстов: конкретно эта имплементация сначала подбирает кандидаты через CountVectorizer, а затем ранжирует слова по косинусной близости вектора кандидата и вектора текста. Это, как мне кажется, и является слабостью алгоритма: он предпочитает выделять наиболее длинные ключевые фразы как более похожие и, как следствие, более информативные.

Это самый главный минус выделения клбчевых слов при помощи берта (таким способом), мне кажется, для выделения ключевых слов удачнее было бы смотреть на матрицы внимания, которые по сути и выделяют самые важные слова в тексте, только надо понять, как аггрегировать информацию: брать за ключевые те слова к которым attend больше всего токенов (с самым большим весом) или еще как-то.

\+ из-за костыля с перебором `ngram_range`, чтобы получить хоть какие-то однословные ключевые фразы, выдается очень много схожих сочетаний: `bars`, `energy bars`, `energy bars containing`, `nutrition energy bars` и тд, что тоже неидеально

При этом получается, что выделенные фразы не всегда логичные, и представляют не то чтобы связные куски текста, например `bars less grams`, `energy bars convinient` и тд.

In [52]:
print('TARGET:', dataset.target[0])
print('PRED:', dataset.bert_4[0])

TARGET: {'energy', 'grams', 'contain', 'fat', 'snacks', 'food', 'sugars', 'bars', 'bar', 'saturated', 'calories', 'syrup', 'nutrition', 'saturated fats', 'healthy', 'protein', 'energy bar'}
PRED: {'snacks bar', 'bar protein carbohydrates', 'candy bar', 'nutrition energy', 'energy bars convenient', 'meals snacks bar', 'bars', 'nutrition energy bars', 'bars less grams', 'nutritional', 'energy bars contain', 'nutrition', 'calories bars consumed', 'calories bars', 'bars contain', 'bar protein', 'sugar granola bars', 'snacks', 'bars ingredients', 'bars consumed', 'energy bars', 'afford calories bars', 'energy bars containing', 'bar', 'granola bars'}
